# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.5
Scikit-Learn 1.0.2
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(64, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.40))

#Adding a second convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.40))


#Flattening
classifier.add(Flatten())


#Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.50))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0

In [5]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../dataset_mel_final/train'
parth_dir_test = '../dataset_mel_final/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    class_mode = 'binary'
)

test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size=(150,150),
                                        batch_size =64,
                                        class_mode = 'binary')


Found 1102 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint('best_model3.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 600,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/600
18/18 [==============================] - 9s 293ms/step - loss: 3.4532 - accuracy: 0.5073 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/600
18/18 [==============================] - 4s 224ms/step - loss: 0.6922 - accuracy: 0.5354 - val_loss: 0.6900 - val_accuracy: 0.5000
Epoch 3/600
18/18 [==============================] - 4s 227ms/step - loss: 0.6529 - accuracy: 0.6216 - val_loss: 0.5910 - val_accuracy: 0.8066
Epoch 4/600
18/18 [==============================] - 4s 224ms/step - loss: 0.5462 - accuracy: 0.7314 - val_loss: 0.4629 - val_accuracy: 0.8431
Epoch 5/600
18/18 [==============================] - 4s 232ms/step - loss: 0.5006 - accuracy: 0.7595 - val_loss: 0.3847 - val_accuracy: 0.8431
Epoch 6/600
18/18 [==============================] - 4s 245ms/step - loss: 0.4693 - accuracy: 0.8013 - val_loss: 0.3961 - val_accuracy: 0.8431
Epoch 7/600
18/18 [==============================] - 4s 234ms/step - loss: 0.4066 - accuracy: 0.8312 - val_loss: 0.2933 - val_accuracy: 0.9124

Epoch 58/600
18/18 [==============================] - 5s 255ms/step - loss: 0.0706 - accuracy: 0.9755 - val_loss: 0.0797 - val_accuracy: 0.9672
Epoch 59/600
18/18 [==============================] - 4s 246ms/step - loss: 0.0817 - accuracy: 0.9773 - val_loss: 0.0799 - val_accuracy: 0.9672
Epoch 60/600
18/18 [==============================] - 4s 242ms/step - loss: 0.0763 - accuracy: 0.9719 - val_loss: 0.0803 - val_accuracy: 0.9635
Epoch 61/600
18/18 [==============================] - 4s 251ms/step - loss: 0.0773 - accuracy: 0.9737 - val_loss: 0.0808 - val_accuracy: 0.9635
Epoch 62/600
18/18 [==============================] - 4s 231ms/step - loss: 0.0739 - accuracy: 0.9728 - val_loss: 0.0812 - val_accuracy: 0.9635
Epoch 63/600
18/18 [==============================] - 4s 218ms/step - loss: 0.0792 - accuracy: 0.9782 - val_loss: 0.0816 - val_accuracy: 0.9635
Epoch 64/600
18/18 [==============================] - 4s 234ms/step - loss: 0.0730 - accuracy: 0.9728 - val_loss: 0.0805 - val_accuracy:

In [10]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [12]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11704), started 1 day, 0:15:18 ago. (Use '!kill 11704' to kill it.)

# Part 3 test the CNN

In [11]:
# Load Model
classifier.load_weights('best_model3.h5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 37, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0

In [14]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Test Loss 0.22910866141319275
Test accuracy 0.9186046719551086
